# 2.2. Advanced NumPy
NumPy is at the base of Python's scientific stack of tools. Its purpose to implement efficient operations on many items in a block of memory. Understanding how it works in detail helps in making efficient use of its flexibility, taking useful shortcuts.

This section covers:
* Anatomy of NumPy arrays, and its consequences. Tips and tricks.
* Universal functions: what, why, and what to do if you want a new one.
* Intergration with other tools: NumPy offers serveral ways to wrap any data in an ndarray, without unneecsary copies.
* Recently added features, and what's in them: PEP 3118 buffers, generalized ufuncs,...

Prerequisites:
* NumPy
* Cython
* Pillow(Python imaging library, used in a couple of examples)

[TOC]

In [1]:
import numpy as np

## 2.2.1 Life of ndarray
### 2.2.1.1 It's
ndarray=
    block of memory + indexing scheme + data type descriptor

* raw data
* how to locate an element
* how to interpret an element
![](http://www.scipy-lectures.org/_images/threefundamental.png)

```C
typedef struct PyArrayObject {
    PyObject_HEAD
    
    /* Block of memory */
    char *data;
    
    /* Data type descriptor */
    PyArray_Descr *descr;
                              
    /* Indexing scheme */
    int nd;
    npy_intp *dimensions;
    npy_intp *strides;
                      
    /* Other stuff */
    PyObject *base;
    int flags;
    PyObject *weakreflist;
                              
} PyArrayObject;
```

### 2.2.1.2 Block of memory

In [6]:
x = np.array([1, 2, 3], dtype=np.int32)
x.data

<read-write buffer for 0x7f1ec548fd00, size 12, offset 0 at 0x7f1ec5420670>

In [7]:
bytes(x.data)

'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'

Memory address of the data:

In [8]:
x.__array_interface__['data'][0]

40452720

In [9]:
x.__array_interface__

{'data': (40452720, False),
 'descr': [('', '<i4')],
 'shape': (3,),
 'strides': None,
 'typestr': '<i4',
 'version': 3}

Reminder: two ndarrays may share the same memory:

In [10]:
x = np.array([1, 2, 3, 4])
y = x[:-1]
x[0] = 9
y

array([9, 2, 3])

In [12]:
# Memory does not need to be owned by an ndarray:
x = b'1234' # The 'b' is for "bytes", necessary in Python3

# x is a string(in Python3 a bytes), we can represent its data as an array of ints:
y = np.frombuffer(x, dtype=np.int8)
y.data, y.base is x

(<read-only buffer for 0x7f1ec542ead0, size 4, offset 0 at 0x7f1ec5441070>,
 True)

In [13]:
y.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : False
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

### 2.2.1.3 Data types
The descriptor
dtype descripes a single item in teh array:
```
type       scalar type of the data, one of:
           int8, int16, float64, et al.(fixed size)
           str, unicode, void(flexible size)
itemsize   size of the data block
byteorder  byte order:big-endian > / little-endian < / not applicable |
fields     sub-dtypes, if it's a structured data type
shape      shape of the array, if it's a sub-array
```

In [15]:
np.dtype(int).type, np.dtype(int).itemsize, np.dtype(int).byteorder

(numpy.int64, 8, '=')

#### Example: reading .wav files
The .wav file header:
```
chunk_id      "RIFF"
chunk_size    4-byte unsigned little-endian integer
format        "WAVE"
fmt_id        "fmt"
fmt_size      4-byte unsigned little-endian integer
audio_fmt     2-byte unsigned little-endian integer
num_channels  2-byte unsigned little-endian integer
sample_rate   4-byte unsigned little-endian integer
byte_rate     4-byte unsigned little-endian integer
block_align   2-byte unsigned little-endian integer
bits_per_sample  2-byte unsigned little-endian integer
data_id       "data"
data_size     4-byte unsigned little-endian integer
```

* 44-byte block of raw data(in the beginning of the file)
* ... followed by data_size bytes of actual sound data

In [16]:
wav_header_dtype = np.dtype([
    ("chunk_id", (bytes, 4)), # flexible-sized scalar type, item size 4
    ("chunk_size", "<u4"),  # little-endian unsigned 32-bit integer
    ("format", "S4"), # 4-byte string
    ("fmt_id", "S4"),
    ("fmt_size", "<u4"),
    ("audio_fmt", "<u2"),
    ("num_channles", "<u2"), # .. more of the same
    ("sample_rate", "<u4"),
    ("byte_rate", "<u4"),
    ("block_align", "<u2"),
    ("bits_per_sample", "<u2"),
    ("data_id", ("S1", (2, 2))),  # sub-array, just for fun!
    ("data_size", "u4"),
    # the sound data itself cannot be represented here: it does not have a fixed size
])

In [17]:
wav_header_dtype['format']

dtype('S4')

In [18]:
wav_header_dtype.fields

dict_proxy({'audio_fmt': (dtype('uint16'), 20),
            'bits_per_sample': (dtype('uint16'), 34),
            'block_align': (dtype('uint16'), 32),
            'byte_rate': (dtype('uint32'), 28),
            'chunk_id': (dtype('S4'), 0),
            'chunk_size': (dtype('uint32'), 4),
            'data_id': (dtype(('S1', (2, 2))), 36),
            'data_size': (dtype('uint32'), 40),
            'fmt_id': (dtype('S4'), 12),
            'fmt_size': (dtype('uint32'), 16),
            'format': (dtype('S4'), 8),
            'num_channles': (dtype('uint16'), 22),
            'sample_rate': (dtype('uint32'), 24)})

In [19]:
wav_header_dtype.fields['format']

(dtype('S4'), 8)

* The first element is the sub-dtype in the structured data, corresponding to the name format
* the second one is its offset(in bytes) from the beginning of the item

### Exercise
Mini-exercise, make a "sparse" dtype by using offsets, and only some of the fields:
```Python
wav_header_dtype = np.dtype(dict(
    names=['format', 'sample_rate', 'data_id'],
    offsets=[offset_1, offset_2, offset_3], # counted from start of structure in bytes
    formates=list of dtypes for each of the fields,
))
```
and use that to read the sample rate, and data_id(as sub-array).

In [24]:
f = open('data/test.wav', 'r')
wav_header = np.fromfile(f, dtype=wav_header_dtype, count=1)
f.close()
print(wav_header)

[('RIFF', 17402, 'WAVE', 'fmt ', 16, 1, 1, 16000, 32000, 2, 16, [['d', 'a'], ['t', 'a']], 17366)]


In [25]:
wav_header['sample_rate']

array([16000], dtype=uint32)

In [26]:
wav_header['data_id']

array([[['d', 'a'],
        ['t', 'a']]], dtype='|S1')

In [27]:
wav_header.shape

(1,)

In [28]:
wav_header['data_id'].shape

(1, 2, 2)

When accessing sub-arrays, the dimensions get added to the end!
**Note**: There are existing modules subh as `wavfile`,`audiolab`, etc. for loading sound data...

### Casting and re-interpretation/views
**casting**
* on assignment
* on array construction
* on arithmetic
* etc.
* and manually: .astype(dtype)

**data re-interpretation**
* manually: .view(dtype)

### Casting
* Casting in arithmetic, in nutshell:
    * only type(not value!) of operands matters
    * largest "safe" type able to represent both is picked
    * scalars can "lose" to arrays in some situations
* Casting in general copies data:

In [29]:
x = np.array([1, 2, 3, 4], dtype=np.float)
x

array([1., 2., 3., 4.])

In [32]:
y = x.astype(np.int8)
y

array([1, 2, 3, 4], dtype=int8)

In [34]:
y + 1, y + 256, y + 256.0

(array([2, 3, 4, 5], dtype=int8),
 array([257, 258, 259, 260], dtype=int16),
 array([257., 258., 259., 260.]))

In [35]:
y + np.array([256], dtype=np.int32)

array([257, 258, 259, 260], dtype=int32)

* Casting on setitem: dtype of the array is not changed non item assignment:

In [37]:
y[:] = y + 1.5
y

array([2, 3, 4, 5], dtype=int8)

### Re-interpretation / viewing
* Data block in memory(4 bytes)

    0x01 || 0x02 || 0x03 || 0x04

    * 4 of uint8, OR
    * 4 of int8, OR
    * 2 of int16, OR
    * 1 of int32, OR
    * 1 of float32, OR
    ...
    
How to switch from one to another?
1. Switch the dtype:

In [40]:
x = np.array([1, 2, 3, 4], dtype=np.uint8)
x.dtype = "<i2"
x, 0x0201, 0x0403

(array([ 513, 1027], dtype=int16), 513, 1027)

    0x01  0x02 || 0x03 0x04
2. Create a new view:

In [42]:
y = x.view("<i4")
y, 0x04030201

(array([67305985], dtype=int32), 67305985)

**Note**:
* .view() makes views, does not copy (or alter) the memory block
* only changes the dtype (and adjusts array shape)

In [44]:
x[1] = 5
y, y.base is x

(array([328193], dtype=int32), True)

**Mini-exersice: data re-interpretation**

You have RGBA data in an array:

In [48]:
x = np.zeros((10, 10, 4), dtype=np.int8)
x[:, :, 0] = 1
x[:, :, 1] = 2
x[:, :, 2] = 3
x[:, :, 3] = 4

where the last three dimensions are the G,B and G, and alpha channels.

How to make a (10, 10) structured array with field names 'r','g','b','a' without copying data?

```
y = ...
assert (y['r'] == 1).all()
assert (y['g'] == 2).all()
assert (y['b'] == 3).all()
assert (y['a'] == 4).all()
```

In [54]:
y = np.array([[1, 3], [2, 4]], dtype=np.uint8).transpose()
x = y.copy()
x, y

array([[1, 2],
       [3, 4]], dtype=uint8)

In [56]:
x.view(np.int16), 0x0201, 0x0403

(array([[ 513],
        [1027]], dtype=int16), 513, 1027)

In [57]:
y.view(np.int16), 0x0301, 0x0402

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Changing the shape of an F-contiguous array by descriptor assignment is deprecated. To maintain the Fortran contiguity of a multidimensional Fortran array, use 'a.T.view(...).T' instead
  """Entry point for launching an IPython kernel.


(array([[ 769, 1026]], dtype=int16), 769, 1026)

### 2.2.1.4 Indexing scheme: strides
Main point

The question:

In [59]:
x = np.array([[1, 2, 3],
             [4, 5, 6],
             [7, 8, 9]], dtype=np.int8)
str(x.data)

'\x01\x02\x03\x04\x05\x06\x07\x08\t'

At which in ``x.data`` does the item ``x[1, 2]`` begin?

The answer (in NumPy)
* **strides**: the number of bytes to jump to find the next element
* 1 stride per dimension

In [61]:
x.strides

(3, 1)

In [62]:
byte_offset = 3*1 + 1*2  # to find x[1, 2]
x.flat[byte_offset], x[1, 2]

(6, 6)

#### C and Fortran order

In [64]:
x = np.array([[1, 2, 3], [4, 5, 6]], dtype=np.int16, order='C')
x.strides, str(x.data)

((6, 2), '\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00')